In [1]:
from main import DataImporter
from models.modelhelper import ModelHelper
import numpy as np

data_importer = DataImporter()
data_importer.import_data()

ds_train = data_importer.get_train_data()
ds_test = data_importer.get_test_data()
ds_validation = data_importer.get_validation_data()

In [2]:
ds_train.info()
print('\n')
ds_test.info()
print('\n')
ds_validation.info()
print('\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10269 entries, 0 to 10268
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10269 non-null  object 
 1   label                 10269 non-null  int64  
 2   statement             10269 non-null  object 
 3   subject               10269 non-null  object 
 4   speaker               10269 non-null  object 
 5   job_title             10269 non-null  object 
 6   state_info            10269 non-null  object 
 7   party_affiliation     10269 non-null  object 
 8   barely_true_counts    10269 non-null  float32
 9   false_counts          10269 non-null  float32
 10  half_true_counts      10269 non-null  float32
 11  mostly_true_counts    10269 non-null  float32
 12  pants_on_fire_counts  10269 non-null  float32
 13  context               10269 non-null  object 
dtypes: float32(5), int64(1), object(8)
memory usage: 922.7+ KB


<class 'p

In [3]:
# Drop specified columns from each dataset
columns_to_drop = ['id', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'context']

ds_train = ds_train.drop(columns=columns_to_drop)
ds_test = ds_test.drop(columns=columns_to_drop) 
ds_validation = ds_validation.drop(columns=columns_to_drop)

# Display results
print("Training Dataset:")
display(ds_train.head())
print("\nTest Dataset:") 
display(ds_test.head())
print("\nValidation Dataset:")
display(ds_validation.head())

Training Dataset:


,label,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts
0,0,Says the Annies List political group supports ...,0.0,1.0,0.0,0.0,0.0
1,1,When did the decline of coal start? It started...,0.0,0.0,1.0,1.0,0.0
2,2,"Hillary Clinton agrees with John McCain ""by vo...",70.0,71.0,160.0,163.0,9.0
3,0,Health care reform legislation is likely to ma...,7.0,19.0,3.0,5.0,44.0
4,1,The economic turnaround started at the end of ...,15.0,9.0,20.0,19.0,2.0



Test Dataset:


,label,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts
0,3,Building a wall on the U.S.-Mexico border will...,30.0,30.0,42.0,23.0,18.0
1,0,Wisconsin is on pace to double the number of l...,2.0,1.0,0.0,0.0,0.0
2,0,Says John McCain has done nothing to help the ...,63.0,114.0,51.0,37.0,61.0
3,1,Suzanne Bonamici supports a plan that will cut...,1.0,1.0,3.0,1.0,1.0
4,5,When asked by a reporter whether hes at the ce...,5.0,7.0,2.0,2.0,7.0



Validation Dataset:


,label,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts
0,4,We have less Americans working now than in the...,1.0,0.0,1.0,0.0,0.0
1,5,"When Obama was sworn into office, he DID NOT u...",11.0,43.0,8.0,5.0,105.0
2,0,Says Having organizations parading as being so...,0.0,1.0,1.0,1.0,0.0
3,1,Says nearly half of Oregons children are poor.,0.0,1.0,1.0,1.0,0.0
4,1,On attacks by Republicans that various program...,70.0,71.0,160.0,163.0,9.0


In [ ]:
def new_false(ds_train, ds_test, ds_validation):
    """
    Combines the 'false_counts' and 'pants_on_fire_counts' columns into a single 'false_counts' column,
    and combines the 'barely_true' column into the 'half_true' column.
    Drops the 'pants_on_fire_counts' and 'barely_true' columns from each dataset.

    Parameters:
        ds_train (pd.DataFrame): Training dataset
        ds_test (pd.DataFrame): Test dataset
        ds_validation (pd.DataFrame): Validation dataset

    Returns:
        tuple: Updated (ds_train, ds_test, ds_validation) DataFrames
    """
    for ds in [ds_train, ds_test, ds_validation]:
        # Combine false_counts and pants_on_fire_counts
        ds['false_counts'] = ds['false_counts'] + ds['pants_on_fire_counts']
        ds.drop('pants_on_fire_counts', axis=1, inplace=True)
        
        # Combine barely_true into half_true_counts
        ds['half_true_counts'] = ds['half_true_counts'] + ds['barely_true_counts']
        ds.drop('barely_true_counts', axis=1, inplace=True)
        
    return ds_train, ds_test, ds_validation

In [5]:
ds_train, ds_test, ds_validation = new_false(ds_train, ds_test, ds_validation)

KeyError: 'half_true'

In [ ]:
# Display results
print("Training Dataset:")
display(ds_train.head())
print("\nTest Dataset:") 
display(ds_test.head())
print("\nValidation Dataset:")
display(ds_validation.head())

In [ ]:
# Find longest statement
longest_statement = ds_train.loc[ds_train['statement'].str.len().idxmax(), 'statement']
# Get the labels for the longest statement
longest_statement_label = ds_train.loc[ds_train['statement'].str.len().idxmax(), 'label']

# Get word count by splitting on whitespace and counting tokens
word_count = len(longest_statement.split())


print(f"Longest statement ({len(longest_statement)} characters):")
print(longest_statement)
print(f'Longest statement word count: {word_count}')
print(f"Longest statement label: {longest_statement_label}")

In [ ]:
# Initialize ModelHelper
model_helper = ModelHelper()

In [ ]:
#Create Vectorizer
model_helper.create_vectorizer(ds_train['statement'], max_sequence_length=60)

# Preprocess text data
train_sequences = model_helper.preprocess_text(ds_train['statement'].tolist())
test_sequences = model_helper.preprocess_text(ds_test['statement'].tolist())
val_sequences = model_helper.preprocess_text(ds_validation['statement'].tolist())

In [ ]:
# Get truthfulness columns
truthfulness_columns = model_helper.truthfulness_columns
truthfulness_columns.remove('pants_on_fire_counts')

In [ ]:
# Get the raw count values for training
train_labels = model_helper.normalize_counts(ds_train)
test_labels = model_helper.normalize_counts(ds_test)
val_labels = model_helper.normalize_counts(ds_validation)

In [ ]:
# Create text classification model
vocab_size = 10000  # Matches max_tokens in preprocess_text
embedding_dim = 100
max_sequence_length = 200
num_classes = len(truthfulness_columns)  # Number of truthfulness categories

model = model_helper.create_text_classification_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim, 
    max_sequence_length=max_sequence_length,
    num_classes=num_classes
)

In [ ]:
# Prepare datasets
train_dataset, val_dataset, test_dataset = model_helper.prepare_datasets(
    train_sequences=train_sequences,
    train_labels=train_labels,
    val_sequences=val_sequences,
    val_labels=val_labels,
    test_sequences=test_sequences,
    test_labels=test_labels,
    batch_size=32
)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir models/logs/fit

# Train the model using ModelHelper's train_model method
history = model_helper.train_model(
    model=model,
    train_data=train_dataset,
    validation_data=val_dataset,
    epochs=15,
    batch_size=32
)

In [ ]:
# Evaluate on test set
test_metrics = model.evaluate(test_dataset)
print(test_metrics)

In [ ]:
# Save the model
model_helper.save_model(model, "text_classification_model_final2")

In [ ]:
# Load the saved model
loaded_model = model_helper.load_model("text_classification_model_final2")
test_string = 'Barbara Buono by the numbers: As a Trenton politician, she voted to raise taxes 154 times. Under her, property taxes up 70 percent. Backed a 16 percent sales tax increase. Utilities, nursing homes, cell phones, parking lots, lottery wins, gyms She taxed them all. Architect of Corzines budget, she drove New Jersey $2 billion into debt. Barbara Buono by the numbers: taking New Jersey backwards.'
print(model_helper.preprocess_text(test_string))
prediction = loaded_model.predict(model_helper.preprocess_text(test_string))
print(prediction)

In [ ]:
# Make predictions on test dataset
print(test_dataset)
predictions = loaded_model.predict(test_dataset)
# Convert predictions to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Print sample predictions
print("\nSample predictions:")
for i in range(5):
    print(f"Example {i+1}:")
    print(f"Predicted probabilities: {predictions[i]}")
    print(f"Predicted class: {predicted_classes[i]}")
    print(f"Actual class: {np.argmax(test_labels[i])}\n")


In [ ]:
models_list = model_helper.list_models()
print(models_list)